In [2]:
import json
import pandas as pd
from datetime import datetime
from listas import charlson_dict, delirium_list, dementia_list, depresyndr_list, uriincont_list, fecincont_list, pressulc_list, immob_list, conf_list, osteopor_list, sarcopenia_list, sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list, ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list, psicosis_list

from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtenir_ultim_resultat, obtenir_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

taula = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


In [3]:
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,urea,FGE MDRD,FGE CDK-EPI,pes més antic,pes més nou,data pes més antic,data primer mecvv,pes coincident primer mecvv,perdua pes entre ingressos,perdua pes total
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,25.7,>60,>90,70,53.7,2022-07-06,None,None,None,16.3
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,2020-12-04,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,None,None,None,0.0


In [4]:
AMB_PA = taula[taula["PA_diagnosticada"] == 1.0]
AMB_PA_MECVV = taula[
    (taula["P_diagnosticada"] == 1.0) & (taula["Dias_totales_ingresado"] < 30) & (taula["Disfagia_mecvvs"] == 1.0) & (taula["alteracioEficacia_mecvvs"] == 1.0) |
    (taula["P_diagnosticada"] == 1.0) & (taula["Dias_totales_ingresado"] < 30) & (taula["Disfagia_mecvvs"] == 1.0) & (taula["alteracioSeguretat_mecvvs"] == 1.0)]   
SENSE_PA = taula[
          (taula["P_diagnosticada"] == 1.0) |
          (taula["P_diagnosticada"] == 1.0) & (taula["Disfagia_mecvvs"] == 1.0) & (taula["alteracioEficacia_mecvvs"] == 1.0) & (taula["Dias_totales_ingresado"] > 30) |
          (taula["P_diagnosticada"] == 1.0) & (taula["Disfagia_mecvvs"] == 1.0) & (taula["alteracioSeguretat_mecvvs"] == 1.0) & (taula["Dias_totales_ingresado"] > 30)]

# Síndromes geriàtrics

## Index de Charlson

In [5]:
promedio_charlson = AMB_PA['charlson'].mean()
desviacion_estandar_charlson = AMB_PA['charlson'].std()

resultado = f"{promedio_charlson:.2f} ± {desviacion_estandar_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultado)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 10.24 ± 10.73


In [6]:
promedio_charlson = AMB_PA_MECVV['charlson'].mean()
desviacion_estandar_charlson = AMB_PA_MECVV['charlson'].std()

resultado = f"{promedio_charlson:.2f} ± {desviacion_estandar_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultado)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 13.60 ± 10.91


In [7]:
promedio_charlson = SENSE_PA['charlson'].mean()
desviacion_estandar_charlson = SENSE_PA['charlson'].std()

resultado = f"{promedio_charlson:.2f} ± {desviacion_estandar_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultado)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 11.73 ± 13.27


#### Pvalor

##### Normalitat

In [8]:
stat, p_value = shapiro(AMB_PA['charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7994
Valor p = 0.0020
El nombre de files no segueix una distribució normal.


In [9]:
stat, p_value = shapiro(AMB_PA_MECVV['charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9380
Valor p = 0.5307
El nombre de files segueix una distribució normal


In [10]:
stat, p_value = shapiro(SENSE_PA['charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7879
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [11]:
stat, p_value_12 = mannwhitneyu(AMB_PA['charlson'], AMB_PA_MECVV['charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.4197045980559335


In [12]:
stat, p_value_13 = mannwhitneyu(AMB_PA['charlson'], SENSE_PA['charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.8657225966560429


In [13]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['charlson'], SENSE_PA['charlson'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.4062768013065785


## Caigudes prèvies

In [14]:
caprev_AMB_PA= (AMB_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA:.2f}%")

Percentatge de caigudes prèvies: 5.88%


In [15]:
caprev_AMB_PA_MECVV= (AMB_PA_MECVV['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA_MECVV:.2f}%")

Percentatge de caigudes prèvies: 0.00%


In [16]:
caprev_SENSE_PA= (SENSE_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_SENSE_PA:.2f}%")

Percentatge de caigudes prèvies: 6.12%


#### Pvalor

##### Normalitat

In [17]:
stat, p_value = shapiro(AMB_PA['caidas_previas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [18]:
stat, p_value = shapiro(AMB_PA_MECVV['caidas_previas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [19]:
stat, p_value = shapiro(SENSE_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [20]:
stat, p_value_12 = mannwhitneyu(AMB_PA['caidas_previas'], AMB_PA_MECVV['caidas_previas'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.49002535950179893


In [21]:
stat, p_value_13 = mannwhitneyu(AMB_PA['caidas_previas'], SENSE_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.9858477456716862


In [22]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['caidas_previas'], SENSE_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.44143809205435813


## Deliris

In [23]:
del_AMB_PA= (AMB_PA['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA:.2f}%")

Percentatge de deliris: 23.53%


In [24]:
del_AMB_PA_MECVV= (AMB_PA_MECVV['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA_MECVV:.2f}%")

Percentatge de deliris: 40.00%


In [25]:
del_SENSE_PA= (SENSE_PA['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_SENSE_PA:.2f}%")

Percentatge de deliris: 18.37%


#### Pvalor

##### Normalitat

In [26]:
stat, p_value = shapiro(AMB_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [27]:
stat, p_value = shapiro(AMB_PA_MECVV['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6405
Valor p = 0.0002
El nombre de files no segueix una distribució normal.


In [28]:
stat, p_value = shapiro(SENSE_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [29]:
stat, p_value_12 = mannwhitneyu(AMB_PA['delirios'], AMB_PA_MECVV['delirios'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.39174655228054167


In [30]:
stat, p_value_13 = mannwhitneyu(AMB_PA['delirios'], SENSE_PA['delirios'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.6549000779898075


In [31]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['delirios'], SENSE_PA['delirios'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.13960802912048415


## Demència

In [32]:
dem_AMB_PA= (AMB_PA['demencia'] == 1.0).mean() * 100
print(f"Percentatge de demencia: {dem_AMB_PA:.2f}%")

Percentatge de demencia: 47.06%


In [33]:
dem_AMB_PA_MECVV= (AMB_PA_MECVV['demencia'] == 1.0).mean() * 100
print(f"Percentatge de demencia: {dem_AMB_PA_MECVV:.2f}%")

Percentatge de demencia: 40.00%


In [34]:
dem_SENSE_PA= (SENSE_PA['demencia'] == 1.0).mean() * 100
print(f"Percentatge de demencia: {dem_SENSE_PA:.2f}%")

Percentatge de demencia: 30.61%


#### Pvalor

##### Normalitat

In [35]:
stat, p_value = shapiro(AMB_PA['demencia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6418
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [36]:
stat, p_value = shapiro(AMB_PA_MECVV['demencia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6405
Valor p = 0.0002
El nombre de files no segueix una distribució normal.


In [37]:
stat, p_value = shapiro(SENSE_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [38]:
stat, p_value_12 = mannwhitneyu(AMB_PA['demencia'], AMB_PA_MECVV['demencia'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.7484826918143946


In [39]:
stat, p_value_13 = mannwhitneyu(AMB_PA['demencia'], SENSE_PA['demencia'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.22703869678020527


In [40]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['demencia'], SENSE_PA['demencia'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.5743933608939569


## Síndrome depressiu

In [41]:
depre_AMB_PA= (AMB_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA:.2f}%")

Percentatge de síndrome depressiu: 23.53%


In [42]:
depre_AMB_PA_MECVV= (AMB_PA_MECVV['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA_MECVV:.2f}%")

Percentatge de síndrome depressiu: 0.00%


In [43]:
depre_SENSE_PA= (SENSE_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_SENSE_PA:.2f}%")

Percentatge de síndrome depressiu: 14.29%


#### Pvalor

##### Normalitat

In [44]:
stat, p_value = shapiro(AMB_PA['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [45]:
stat, p_value = shapiro(AMB_PA_MECVV['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [46]:
stat, p_value = shapiro(SENSE_PA['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4168
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [47]:
stat, p_value_12 = mannwhitneyu(AMB_PA['sindrome_depresivo'], AMB_PA_MECVV['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.11180527770715322


In [48]:
stat, p_value_13 = mannwhitneyu(AMB_PA['sindrome_depresivo'], SENSE_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.3880726918610017


In [49]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['sindrome_depresivo'], SENSE_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.21340049774345338


## Incontinència urinària

In [50]:
ic_AMB_PA= (AMB_PA['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_AMB_PA:.2f}%")

Percentatge de incontinència urinària: 29.41%


In [51]:
ic_AMB_PA_MECVV= (AMB_PA_MECVV['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_AMB_PA_MECVV:.2f}%")

Percentatge de incontinència urinària: 30.00%


In [52]:
ic_SENSE_PA= (SENSE_PA['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_SENSE_PA:.2f}%")

Percentatge de incontinència urinària: 14.29%


#### Pvalor

##### Normalitat

In [53]:
stat, p_value = shapiro(AMB_PA['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [54]:
stat, p_value = shapiro(AMB_PA_MECVV['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5942
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [55]:
stat, p_value = shapiro(SENSE_PA['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4168
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [56]:
stat, p_value_12 = mannwhitneyu(AMB_PA['incont_uri'], AMB_PA_MECVV['incont_uri'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 1.0


In [57]:
stat, p_value_13 = mannwhitneyu(AMB_PA['incont_uri'], SENSE_PA['incont_uri'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.17015925929193942


In [58]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['incont_uri'], SENSE_PA['incont_uri'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.237514563226218


## Incontinència fecal 

In [59]:
ifec_AMB_PA= (AMB_PA['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_AMB_PA:.2f}%")

Percentatge de incontinència fecal: 29.41%


In [60]:
ifec_AMB_PA_MECVV= (AMB_PA_MECVV['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_AMB_PA_MECVV:.2f}%")

Percentatge de incontinència fecal: 10.00%


In [61]:
ifec_SENSE_PA= (SENSE_PA['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_SENSE_PA:.2f}%")

Percentatge de incontinència fecal: 12.24%


#### Pvalor

##### Normalitat

In [62]:
stat, p_value = shapiro(AMB_PA['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [63]:
stat, p_value = shapiro(AMB_PA_MECVV['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [64]:
stat, p_value = shapiro(SENSE_PA['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3844
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [65]:
stat, p_value_12 = mannwhitneyu(AMB_PA['incont_fec'], AMB_PA_MECVV['incont_fec'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.2649031747093591


In [66]:
stat, p_value_13 = mannwhitneyu(AMB_PA['incont_fec'], SENSE_PA['incont_fec'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.10681676423525503


In [67]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['incont_fec'], SENSE_PA['incont_fec'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.8568943909879498


## Úlceres de pressió

In [68]:
up_AMB_PA= (AMB_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA:.2f}%")

Percentatge de úlceres de pressió: 5.88%


In [69]:
up_AMB_PA_MECVV= (AMB_PA_MECVV['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA_MECVV:.2f}%")

Percentatge de úlceres de pressió: 0.00%


In [70]:
up_SENSE_PA= (SENSE_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_SENSE_PA:.2f}%")

Percentatge de úlceres de pressió: 2.04%


#### Pvalor

##### Normalitat

In [71]:
stat, p_value = shapiro(AMB_PA['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [72]:
stat, p_value = shapiro(AMB_PA_MECVV['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [73]:
stat, p_value = shapiro(SENSE_PA['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1273
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [74]:
stat, p_value_12 = mannwhitneyu(AMB_PA['ulceras_presion'], AMB_PA_MECVV['ulceras_presion'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.49002535950179893


In [75]:
stat, p_value_13 = mannwhitneyu(AMB_PA['ulceras_presion'], SENSE_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.4440308448923743


In [76]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['ulceras_presion'], SENSE_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.6843175494727258


## Immobilitat

In [77]:
imm_AMB_PA= (AMB_PA['immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA:.2f}%")

Percentatge d'immobilitat : 0.00%


In [78]:
imm_AMB_PA_MECVV= (AMB_PA_MECVV['immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA_MECVV:.2f}%")

Percentatge d'immobilitat : 0.00%


In [79]:
imm_SENSE_PA= (SENSE_PA['immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_SENSE_PA:.2f}%")

Percentatge d'immobilitat : 0.00%


#### Pvalor

##### Normalitat

In [80]:
stat, p_value = shapiro(AMB_PA['immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [81]:
stat, p_value = shapiro(AMB_PA_MECVV['immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [82]:
stat, p_value = shapiro(SENSE_PA['immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [83]:
stat, p_value_12 = ttest_ind(AMB_PA['immobilitat'], AMB_PA_MECVV['immobilitat'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [84]:
stat, p_value_13 = ttest_ind(AMB_PA['immobilitat'], SENSE_PA['immobilitat'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [85]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['immobilitat'], SENSE_PA['immobilitat'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


## Confusió

In [86]:
conf_AMB_PA= (AMB_PA['confusio'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA:.2f}%")

Percentatge de confusió : 23.53%


In [87]:
conf_AMB_PA_MECVV= (AMB_PA_MECVV['confusio'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA_MECVV:.2f}%")

Percentatge de confusió : 40.00%


In [88]:
conf_SENSE_PA= (SENSE_PA['confusio'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_SENSE_PA:.2f}%")

Percentatge de confusió : 18.37%


#### Pvalor

##### Normalitat

In [89]:
stat, p_value = shapiro(AMB_PA['confusio'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [90]:
stat, p_value = shapiro(AMB_PA_MECVV['confusio'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6405
Valor p = 0.0002
El nombre de files no segueix una distribució normal.


In [91]:
stat, p_value = shapiro(SENSE_PA['confusio'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [92]:
stat, p_value_12 = mannwhitneyu(AMB_PA['confusio'], AMB_PA_MECVV['confusio'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.39174655228054167


In [93]:
stat, p_value_13 = mannwhitneyu(AMB_PA['confusio'], SENSE_PA['confusio'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.6549000779898075


In [94]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['confusio'], SENSE_PA['confusio'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.13960802912048415


## Osteoporosi

In [95]:
ost_AMB_PA= (AMB_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA:.2f}%")

Percentatge d'osteoporosi: 0.00%


In [96]:
ost_AMB_PA_MECVV= (AMB_PA_MECVV['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA_MECVV:.2f}%")

Percentatge d'osteoporosi: 0.00%


In [97]:
ost_SENSE_PA= (SENSE_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_SENSE_PA:.2f}%")

Percentatge d'osteoporosi: 0.00%


#### Pvalor

##### Normalitat

In [98]:
stat, p_value = shapiro(AMB_PA['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [99]:
stat, p_value = shapiro(AMB_PA_MECVV['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [100]:
stat, p_value = shapiro(SENSE_PA['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [101]:
stat, p_value_12 = ttest_ind(AMB_PA['osteoporosis'], AMB_PA_MECVV['osteoporosis'], equal_var= False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [102]:
stat, p_value_13 = ttest_ind(AMB_PA['osteoporosis'], SENSE_PA['osteoporosis'], equal_var= False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [103]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['osteoporosis'], SENSE_PA['osteoporosis'], equal_var= False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


## Sarcopènia

In [104]:
sarc_AMB_PA= (AMB_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA:.2f}%")

Percentatge de sarcopènia: 0.00%


In [105]:
sarc_AMB_PA_MECVV= (AMB_PA_MECVV['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA_MECVV:.2f}%")

Percentatge de sarcopènia: 10.00%


In [106]:
sarc_SENSE_PA= (SENSE_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_SENSE_PA:.2f}%")

Percentatge de sarcopènia: 4.08%


#### Pvalor

##### Normalitat

In [107]:
stat, p_value = shapiro(AMB_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [108]:
stat, p_value = shapiro(AMB_PA_MECVV['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [109]:
stat, p_value = shapiro(SENSE_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2008
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [110]:
stat, p_value_12 = mannwhitneyu(AMB_PA['sarcopenia'], AMB_PA_MECVV['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.21976847950216105


In [111]:
stat, p_value_13 = mannwhitneyu(AMB_PA['sarcopenia'], SENSE_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.41519527301502746


In [112]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['sarcopenia'], SENSE_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.45734646700671056


## Trastorns del son

In [113]:
ts_AMB_PA= (AMB_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


In [114]:
ts_AMB_PA_MECVV= (AMB_PA_MECVV['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA_MECVV:.2f}%")

Percentatge de trastorns del son: 0.00%


In [115]:
ts_SENSE_PA= (SENSE_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_SENSE_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


#### Pvalor

##### Normalitat

In [116]:
stat, p_value = shapiro(AMB_PA['problsueño'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [117]:
stat, p_value = shapiro(AMB_PA_MECVV['problsueño'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [118]:
stat, p_value = shapiro(SENSE_PA['problsueño'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [119]:
stat, p_value_12 = ttest_ind(AMB_PA['problsueño'], AMB_PA_MECVV['problsueño'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [120]:
stat, p_value_13 = ttest_ind(AMB_PA['problsueño'], SENSE_PA['problsueño'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [121]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['problsueño'], SENSE_PA['problsueño'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


## Dolor crònic

In [122]:
dc_AMB_PA= (AMB_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


In [123]:
dc_AMB_PA_MECVV= (AMB_PA_MECVV['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA_MECVV:.2f}%")

Percentatge de dolor crònic: 0.00%


In [124]:
dc_SENSE_PA= (SENSE_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_SENSE_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


#### Pvalor

##### Normalitat

In [125]:
stat, p_value = shapiro(AMB_PA['dolor_cron'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [126]:
stat, p_value = shapiro(AMB_PA_MECVV['dolor_cron'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [127]:
stat, p_value = shapiro(SENSE_PA['dolor_cron'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [128]:
stat, p_value_12 = ttest_ind(AMB_PA['dolor_cron'], AMB_PA_MECVV['dolor_cron'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [129]:
stat, p_value_13 = ttest_ind(AMB_PA['dolor_cron'], SENSE_PA['dolor_cron'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [130]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['dolor_cron'], SENSE_PA['dolor_cron'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


## Iatrogènic

In [131]:
iatr_AMB_PA= (AMB_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


In [132]:
iatr_AMB_PA_MECVV= (AMB_PA_MECVV['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA_MECVV:.2f}%")

Percentatge de iatrogènic: 0.00%


In [133]:
iatr_SENSE_PA= (SENSE_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_SENSE_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


#### Pvalor

##### Normalitat

In [134]:
stat, p_value = shapiro(AMB_PA['iatrogenico'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [135]:
stat, p_value = shapiro(AMB_PA_MECVV['iatrogenico'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [136]:
stat, p_value = shapiro(SENSE_PA['iatrogenico'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [137]:
stat, p_value_12 = ttest_ind(AMB_PA['iatrogenico'], AMB_PA_MECVV['iatrogenico'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [138]:
stat, p_value_13 = ttest_ind(AMB_PA['iatrogenico'], SENSE_PA['iatrogenico'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [139]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['iatrogenico'], SENSE_PA['iatrogenico'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


## Restrenyiment

In [140]:
est_AMB_PA= (AMB_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


In [141]:
est_AMB_PA_MECVV= (AMB_PA_MECVV['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA_MECVV:.2f}%")

Percentatge de restrenyiment: 0.00%


In [142]:
est_SENSE_PA= (SENSE_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_SENSE_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


#### Pvalor

##### Normalitat

In [143]:
stat, p_value = shapiro(AMB_PA['estreñimiento'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [144]:
stat, p_value = shapiro(AMB_PA_MECVV['estreñimiento'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [145]:
stat, p_value = shapiro(SENSE_PA['estreñimiento'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [146]:
stat, p_value_12 = ttest_ind(AMB_PA['estreñimiento'], AMB_PA_MECVV['estreñimiento'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [147]:
stat, p_value_13 = ttest_ind(AMB_PA['estreñimiento'], SENSE_PA['estreñimiento'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [148]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['estreñimiento'], SENSE_PA['estreñimiento'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


# Historial mèdic/comorbilitats

## Malalties CV

In [149]:
cv_AMB_PA= (AMB_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA:.2f}%")

Percentatge de problemes CV: 5.88%


In [150]:
cv_AMB_PA_MECVV= (AMB_PA_MECVV['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA_MECVV:.2f}%")

Percentatge de problemes CV: 10.00%


In [151]:
cv_SENSE_PA= (SENSE_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_SENSE_PA:.2f}%")

Percentatge de problemes CV: 6.12%


#### Pvalor

##### Normalitat 

In [152]:
stat, p_value = shapiro(AMB_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [153]:
stat, p_value = shapiro(AMB_PA_MECVV['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [154]:
stat, p_value = shapiro(SENSE_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2580
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [155]:
stat, p_value_12 = mannwhitneyu(AMB_PA['CV'], AMB_PA_MECVV['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.7400102360583588


In [156]:
stat, p_value_13 = mannwhitneyu(AMB_PA['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.9858477456716862


In [157]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.6762982127217047


## Malalties del cor

In [158]:
cora_AMB_PA= (AMB_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_AMB_PA:.2f}%")

Percentatge de malalties de cor: 11.76%


In [159]:
cora_AMB_PA_MECVV= (AMB_PA_MECVV['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_AMB_PA_MECVV:.2f}%")

Percentatge de malalties de cor: 10.00%


In [160]:
cora_SENSE_PA= (SENSE_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_SENSE_PA:.2f}%")

Percentatge de malalties de cor: 14.29%


#### Pvalor

##### Normalitat

In [161]:
stat, p_value = shapiro(AMB_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3853
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [162]:
stat, p_value = shapiro(AMB_PA_MECVV['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [163]:
stat, p_value = shapiro(SENSE_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4168
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [164]:
stat, p_value_12 = mannwhitneyu(AMB_PA['probl_corazon'], AMB_PA_MECVV['probl_corazon'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.9265568396984611


In [165]:
stat, p_value_13 = mannwhitneyu(AMB_PA['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.9858477456716862


In [166]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.6762982127217047


## Malalties neurodegeneratives

In [167]:
nd_AMB_PA= (AMB_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA:.2f}%")

Percentatge de malalties neurodegeneratives: 0.00%


In [168]:
nd_AMB_PA_MECVV= (AMB_PA_MECVV['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA_MECVV:.2f}%")

Percentatge de malalties neurodegeneratives: 10.00%


In [169]:
nd_SENSE_PA= (SENSE_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_SENSE_PA:.2f}%")

Percentatge de malalties neurodegeneratives: 4.08%


#### Pvalor

##### Normalitat

In [170]:
stat, p_value = shapiro(AMB_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [171]:
stat, p_value = shapiro(AMB_PA_MECVV['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [172]:
stat, p_value = shapiro(SENSE_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2008
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [173]:
stat, p_value_12 = mannwhitneyu(AMB_PA['neurodegenerativas'], AMB_PA_MECVV['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.21976847950216105


In [174]:
stat, p_value_13 = mannwhitneyu(AMB_PA['neurodegenerativas'], SENSE_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.41519527301502746


In [175]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['neurodegenerativas'], SENSE_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.45734646700671056


## Diabetes mellitus

In [176]:
dm_AMB_PA= (AMB_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_AMB_PA:.2f}%")

Percentatge de diabetes mellitus: 23.53%


In [177]:
dm_AMB_PA_MECVV= (AMB_PA_MECVV['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_AMB_PA_MECVV:.2f}%")

Percentatge de diabetes mellitus: 10.00%


In [178]:
dm_SENSE_PA= (SENSE_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_SENSE_PA:.2f}%")

Percentatge de diabetes mellitus: 26.53%


#### Pvalor 

##### Normalitat

In [179]:
stat, p_value = shapiro(AMB_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [180]:
stat, p_value = shapiro(AMB_PA_MECVV['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [181]:
stat, p_value = shapiro(SENSE_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5514
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [182]:
stat, p_value_12 = mannwhitneyu(AMB_PA['DM'], AMB_PA_MECVV['DM'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.4120280546315189


In [183]:
stat, p_value_13 = mannwhitneyu(AMB_PA['DM'], SENSE_PA['DM'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PAN", p_value_13)

P-valor entre AMB_PA i SENSE_PAN 0.8163159339942008


In [184]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['DM'], SENSE_PA['DM'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PAN", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PAN 0.2728301710651724


## Hepatopaties

In [185]:
hp_AMB_PA= (AMB_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA:.2f}%")

Percentatge de hepatopaties: 11.76%


In [186]:
hp_AMB_PA_MECVV= (AMB_PA_MECVV['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA_MECVV:.2f}%")

Percentatge de hepatopaties: 10.00%


In [187]:
hp_SENSE_PA= (SENSE_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_SENSE_PA:.2f}%")

Percentatge de hepatopaties: 6.12%


#### Pvalor

##### Normalitat

In [188]:
stat, p_value = shapiro(AMB_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3853
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [189]:
stat, p_value = shapiro(AMB_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [190]:
stat, p_value = shapiro(SENSE_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2580
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [191]:
stat, p_value_12 = mannwhitneyu(AMB_PA['hepatopatias'], AMB_PA_MECVV['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.9265568396984611


In [192]:
stat, p_value_13 = mannwhitneyu(AMB_PA['hepatopatias'], SENSE_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.4618706673466527


In [193]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['hepatopatias'], SENSE_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.6762982127217047


## Neoplàsia

In [194]:
neopl_AMB_PA= (AMB_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_AMB_PA:.2f}%")

Percentatge de neoplàsia: 0.00%


In [195]:
neopl_AMB_PA_MECVV= (AMB_PA_MECVV['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_AMB_PA_MECVV:.2f}%")

Percentatge de neoplàsia: 0.00%


In [196]:
neopl_SENSE_PA= (SENSE_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_SENSE_PA:.2f}%")

Percentatge de neoplàsia: 6.12%


#### Pvalor

##### Normalitat

In [197]:
stat, p_value = shapiro(AMB_PA['neoplasias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [198]:
stat, p_value = shapiro(AMB_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [199]:
stat, p_value = shapiro(SENSE_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2580
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [200]:
stat, p_value_12 = mannwhitneyu(AMB_PA['neoplasias'], AMB_PA_MECVV['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 1.0


In [201]:
stat, p_value_13 = mannwhitneyu(AMB_PA['neoplasias'], SENSE_PA['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_12)

P-valor entre AMB_PA i SENSE_PA 1.0


In [202]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['neoplasias'], SENSE_PA['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.44143809205435813


## Malaltia crònica renal (creatinina > 1.5mg/dL)

Donat que aquesta columna és de tipus objecte, primer la passaré a tipus numèric per poder operar amb ella

In [203]:
AMB_PA.loc[:, 'creatinina'] = pd.to_numeric(AMB_PA['creatinina'], errors='coerce')
AMB_PA_MECVV.loc[:, 'creatinina'] = pd.to_numeric(AMB_PA_MECVV['creatinina'], errors='coerce')
SENSE_PA.loc[:, 'creatinina'] = pd.to_numeric(SENSE_PA['creatinina'], errors='coerce')

In [204]:
ecr_AMB_PA= (AMB_PA['creatinina'] >= 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [205]:
ecr_AMB_PA_MECVV= (AMB_PA_MECVV['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA_MECVV:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [206]:
ecr_SENSE_PA= (SENSE_PA['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_SENSE_PA:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 6.12%


#### Pvalor

##### Normalitat

In [207]:
stat, p_value = shapiro(AMB_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9549
Valor p = 0.7439
El nombre de files segueix una distribució normal


In [208]:
stat, p_value = shapiro(AMB_PA_MECVV['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9815
Valor p = 0.9110
El nombre de files segueix una distribució normal


In [209]:
stat, p_value = shapiro(SENSE_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7235
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [210]:
stat, p_value_12 = ttest_ind(AMB_PA['creatinina'], AMB_PA_MECVV['creatinina'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [211]:
stat, p_value_13 = mannwhitneyu(AMB_PA['creatinina'], SENSE_PA['creatinina'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [212]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['creatinina'], SENSE_PA['creatinina'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


## Fallida renal aguda (o insuficiència renal aguda)

In [213]:
ira_AMB_PA= (AMB_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA:.2f}%")

Percentatge de fallida renal aguda: 23.53%


In [214]:
ira_AMB_PA_MECVV= (AMB_PA_MECVV['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA_MECVV:.2f}%")

Percentatge de fallida renal aguda: 50.00%


In [215]:
ira_SENSE_PA= (SENSE_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_SENSE_PA:.2f}%")

Percentatge de fallida renal aguda: 26.53%


#### Pvalor

##### Normalitat

In [216]:
stat, p_value = shapiro(AMB_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [217]:
stat, p_value = shapiro(AMB_PA_MECVV['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6553
Valor p = 0.0003
El nombre de files no segueix una distribució normal.


In [218]:
stat, p_value = shapiro(SENSE_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5514
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [219]:
stat, p_value_12 = mannwhitneyu(AMB_PA['ARF'], AMB_PA_MECVV['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.17639454522781728


In [220]:
stat, p_value_13 = mannwhitneyu(AMB_PA['ARF'], SENSE_PA['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.8163159339942008


In [221]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['ARF'], SENSE_PA['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_23)

P-valor entre AMB_PA i SENSE_PA 0.14880872192595962


## Marejos/síncope/lipotímia/reacció vasovagal

In [222]:
dizsyn_AMB_PA= (AMB_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [223]:
dizsyn_AMB_PA_MECVV= (AMB_PA_MECVV['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA_MECVV:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [224]:
dizsyn_SENSE_PA= (SENSE_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_SENSE_PA:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 2.04%


#### Pvalor

##### Normalitat

In [225]:
stat, p_value = shapiro(AMB_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [226]:
stat, p_value = shapiro(AMB_PA_MECVV['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [227]:
stat, p_value = shapiro(SENSE_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1273
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [228]:
stat, p_value_12 = ttest_ind(AMB_PA['mareos'], AMB_PA_MECVV['mareos'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [229]:
stat, p_value_12 = mannwhitneyu(AMB_PA['mareos'], AMB_PA_MECVV['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 1.0


In [259]:
stat, p_value_13 = mannwhitneyu(AMB_PA['mareos'], SENSE_PA['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.5793276078637376


In [231]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['mareos'], SENSE_PA['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.6843175494727258


## VIH

In [232]:
vih_AMB_PA= (AMB_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA:.2f}%")

Percentatge de VIH: 0.00%


In [233]:
vih_AMB_PA_MECVV= (AMB_PA_MECVV['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA_MECVV:.2f}%")

Percentatge de VIH: 0.00%


In [234]:
vih_SENSE_PA= (SENSE_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_SENSE_PA:.2f}%")

Percentatge de VIH: 0.00%


#### Pvalor

##### Normalitat

In [235]:
stat, p_value = shapiro(AMB_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [236]:
stat, p_value = shapiro(AMB_PA_MECVV['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [237]:
stat, p_value = shapiro(SENSE_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [238]:
stat, p_value_12 = ttest_ind(AMB_PA['VIH'], AMB_PA_MECVV['VIH'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [239]:
stat, p_value_13 = ttest_ind(AMB_PA['VIH'], SENSE_PA['VIH'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [240]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['VIH'], SENSE_PA['VIH'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


## Psicosi

In [241]:
psi_AMB_PA= (AMB_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {vih_AMB_PA:.2f}%")

Percentatge de psicosi: 0.00%


In [242]:
psi_AMB_PA_MECVV= (AMB_PA_MECVV['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_AMB_PA_MECVV:.2f}%")

Percentatge de psicosi: 0.00%


In [243]:
psi_SENSE_PA= (SENSE_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_SENSE_PA:.2f}%")

Percentatge de psicosi: 0.00%


#### Pvalor 

##### Normalitat

In [244]:
stat, p_value = shapiro(AMB_PA['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [245]:
stat, p_value = shapiro(AMB_PA_MECVV['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [246]:
stat, p_value = shapiro(SENSE_PA['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [247]:
stat, p_value_12 = ttest_ind(AMB_PA['psicosis'], AMB_PA_MECVV['psicosis'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [248]:
stat, p_value_13 = ttest_ind(AMB_PA['psicosis'], SENSE_PA['psicosis'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [249]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['psicosis'], SENSE_PA['psicosis'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


## Deficiències nutricionals

In [250]:
nutri_AMB_PA= (AMB_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [251]:
nutri_AMB_PA_MECVV= (AMB_PA_MECVV['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA_MECVV:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [252]:
nutri_SENSE_PA= (SENSE_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_SENSE_PA:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


#### Pvalor

##### Normalitat

In [253]:
stat, p_value = shapiro(AMB_PA['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [254]:
stat, p_value = shapiro(AMB_PA_MECVV['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [255]:
stat, p_value = shapiro(SENSE_PA['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [256]:
stat, p_value_12 = ttest_ind(AMB_PA['def_nutri'], AMB_PA_MECVV['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [257]:
stat, p_value_13 = ttest_ind(AMB_PA['def_nutri'], SENSE_PA['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [258]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['def_nutri'], SENSE_PA['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan
